In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import maxjoshua as mh
import tensorflow as tf
import sklearn.preprocessing

## Load dataset

In [2]:
import sklearn.datasets
X, y = sklearn.datasets.make_regression(n_samples=1000, n_features=100, n_informative=20, n_targets=3)

In [3]:
X.shape, y.shape, y.max(), y.min()

((1000, 100), (1000, 3), 1116.4693156136254, -900.1283341193364)

## Feature selection

In [4]:
%%time 
indices, values, num_in, num_out = mh.pretrain_submodels(
    sklearn.preprocessing.scale(X), 
    sklearn.preprocessing.scale(y), 
    num_out=64, n_select=3)

0 (38, 48, 88) [0.34481553462340886, -0.022213111722861976, -0.05795348741566542] 0.8755112016888794
1 (2, 65, 80) [0.3098210984522966, 0.029224082309184326, 0.34031906772244597] 0.8680807766479522
2 (2, 16, 60) [0.24840166150232873, -0.0524075663588374, 0.3031005724420399] 0.8842105589345371
CPU times: user 23.3 s, sys: 811 ms, total: 24.1 s
Wall time: 27.2 s


## Training

In [5]:
model = tf.keras.models.Sequential([
    mh.SparseLayerAsEnsemble(
        num_in=num_in, 
        num_out=num_out, 
        sp_indices=indices, 
        sp_values=values,
        sp_trainable=False,
        norm_trainable=True,
    ),
    tf.keras.layers.Dense(
        units=3, use_bias=False,
        # kernel_constraint=tf.keras.constraints.NonNeg()
    ),
    mh.InverseTransformer(
        units=3,
        init_bias=y.mean(), 
        init_scale=y.std()
    )
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=3e-4, beta_1=.9, beta_2=.999, epsilon=1e-7, amsgrad=True),
    loss='mean_squared_error'
)

history = model.fit(X, y, epochs=3)

Epoch 1/3


2022-06-22 15:32:20.410085: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'scale:0' shape=(3,) dtype=float32>
  <tf.Variable 'bias:0' shape=(3,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
32/32 [==============================] - 2s 6ms/step - loss: 79725.2656
Epoch 2/3
32/32 [==============================] - 0s 7ms/step - loss: 73573.5781
Epoch 3/3
32/32 [==============================] - 0s 6ms/step - loss: 68210.8281


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sparse_layer_as_ensemble (S  (None, 64)               592       
 parseLayerAsEnsemble)                                           
                                                                 
 dense (Dense)               (None, 3)                 192       
                                                                 
 inverse_transformer (Invers  (None, 3)                6         
 eTransformer)                                                   
                                                                 
Total params: 790
Trainable params: 398
Non-trainable params: 392
_________________________________________________________________


In [7]:
y_pred = model.predict(X)

32/32 [==============================] - 0s 4ms/step


In [8]:
y_pred

array([[  69.30428 , -106.95536 ,  -82.685486],
       [ -56.667194,  149.03792 ,  -45.18775 ],
       [  63.699768, -153.53983 ,  -24.306787],
       ...,
       [   3.820597,  -97.86668 , -144.86803 ],
       [  35.372498,  -39.756947,   76.79273 ],
       [  31.496029, -123.3764  ,  142.68825 ]], dtype=float32)

In [9]:
y

array([[  86.23141711,   29.82632753,  -21.38341373],
       [  11.61382915,  -32.15349798,    0.82352912],
       [ -72.94832667,  276.13569032,  -67.78254294],
       ...,
       [  33.63370102,  100.9536608 ,  -80.45502987],
       [  -9.51298835, -121.63196117,   53.63461233],
       [ -17.00878224,   52.34750071,  -50.87228869]])

In [10]:
# model.save("test-save")
# model2 = tf.keras.models.load_model("test-save")
# y_pred2 = model2.predict(X)
# y_pred == y_pred2